In [1]:
import os
import time
import random
import wandb
import numpy as np
import gymnasium as gym
import flax.linen as nn
from flax.training.train_state import TrainState
from flax.training.common_utils import onehot

import jax.numpy as jnp
import jax
import optax

from torch.utils.tensorboard import SummaryWriter

In [2]:
def make_env(env_id,seed,idx,capture_video,run_name):
    def thunk():
        if capture_video and idx == 0:
            env = gym.make(env_id,render_mode="rgb_array")
            env = gym.wrappers.RecordVideo(env,f"videos/{run_name}")
        else:
            env = gym.make(env_id)

        env  = gym.wrappers.RecordEpisodeStatistics(env)
        env.action_space.seed(seed)
        return env
    return thunk

In [3]:
class Actor(nn.Module):
    action_dims : int

    @nn.compact
    def __call__(self,input:jnp.ndarray):
        x = nn.Dense(16)(input)
        x = nn.relu(x)
        x = nn.Dense(16)(x)
        x = nn.relu(x)
        x = nn.Dense(self.action_dims)(x)
        probs = nn.softmax(x)
        log_probs = nn.log_softmax(x)

        return probs, log_probs

In [4]:
class Critic(nn.Module):
    hidden_size: int

    @nn.compact
    def __call__(self,input:jnp.ndarray):
        x = nn.Dense(self.hidden_size)(input)
        x = nn.relu(x)
        x = nn.Dense(self.hidden_size)(x)
        x = nn.relu(x)
        x = nn.Dense(1)(x)

        return x

In [5]:
def train(env_id,gamma,episodes,max_termination,seed,num_envs,learning_rate):
    run_name = f"{env_id}__{seed}__{int(time.time())}"
    wandb.init(
        project="a2c-classic-control-benchmark",
        config={
            "env":env_id,
            "gamma":gamma,
            "episodes":episodes,
            "max_termination":max_termination,
            "seed":seed,
            "lr":learning_rate,
        },
        sync_tensorboard=True,
        monitor_gym=True,
        name=run_name
    )
    writer = SummaryWriter(f"runs/{run_name}")
    device = jax.devices("cpu")[0]
    random.seed(seed)
    np.random.seed(seed)
    key = jax.random.PRNGKey(seed)

    env = make_env(env_id,seed,0,True,run_name)()

    obs, _ = env.reset(seed=seed)
    action_dims = env.action_space.n

    actor = Actor(action_dims=action_dims)
    critic = Critic(hidden_size=120)

    actor_state = TrainState.create(
        apply_fn=actor.apply,
        params=jax.device_put(actor.init(key, obs),device),
        tx=optax.adam(learning_rate=learning_rate)
    )

    critic_state = TrainState.create(
        apply_fn=critic.apply,
        params=jax.device_put(critic.init(key, obs),device),
        tx=optax.adam(learning_rate=learning_rate)
    )

    actor.apply = jax.jit(actor.apply)
    critic.apply = jax.jit(critic.apply)

    @jax.jit
    def update(actor_state,critic_state,states,next_states,actions,rewards,dones,gamma):
        def actor_loss_fn(actor_params,critic_params):
            pred_value = critic.apply(critic_params,states).reshape(rewards.shape)
            target_value = rewards + gamma * (critic.apply(critic_params,next_states)).reshape(rewards.shape) * (1 -dones)
            advantage = target_value - pred_value
            advantage = (advantage - advantage.mean()) / (advantage.std() + 1e-8)

            _,logprobs = actor.apply(actor_params,states)
            onehot_actions = onehot(actions,num_classes=logprobs.shape[-1]).reshape(logprobs.shape)
            selected_action_logprobs = jnp.sum(onehot_actions * logprobs,axis=-1)
            actor_loss = -jnp.sum(selected_action_logprobs * advantage)

            return actor_loss

        def critic_loss_fn(critic_params):
            pred_value = critic.apply(critic_params,states)
            target_value = rewards + gamma * critic.apply(critic_params,next_states) * (1 -dones)
            advantage = target_value - pred_value
            # advantage = (advantage - advantage.mean()) / (advantage.std() + 1e-8)
            critic_loss = (advantage**2).mean()

            return critic_loss

        actor_loss,actor_gradients = jax.value_and_grad(actor_loss_fn)(actor_state.params,critic_state.params)
        critic_loss,critic_gradients = jax.value_and_grad(critic_loss_fn)(critic_state.params)

        actor_state = actor_state.apply_gradients(grads=actor_gradients)
        critic_state = critic_state.apply_gradients(grads=critic_gradients)

        return actor_loss,critic_loss,actor_state,critic_state


    for episode in range(episodes):
        states = []
        next_states = []
        rewards = []
        actions = []
        dones = []
        done = False
        obs, _ = env.reset(seed=seed)
        for _ in range(max_termination):
            states.append(obs)
            input = jnp.array(obs)[None,...]
            prob,_ = actor.apply(actor_state.params,jax.device_put(input,device))
            prob = jax.device_get(prob)[0]
            action = np.random.choice(action_dims,p=prob)
            actions.append(action)
            next_state,reward,done,_,_ = env.step(action)
            next_states.append(next_state)
            rewards.append(reward)
            if done:
                dones.append(1)
                break
            dones.append(0)

            obs = next_state

        states = jnp.array(states)
        next_states = jnp.array(next_states)
        rewards = jnp.array(rewards)
        actions = jnp.array(actions)
        dones = jnp.array(dones)

        states = jax.device_put(states,device)
        next_states = jax.device_put(next_states,device)
        rewards = jax.device_put(rewards,device)
        actions = jax.device_put(actions,device)
        dones = jax.device_put(dones,device)

        actor_loss,critic_loss,actor_state,critic_state = update(
            actor_state,
            critic_state,
            states,
            next_states,
            actions,
            rewards,
            dones,
            gamma
        )

        print(f"Episode:{episode}   ActorLoss:{actor_loss} CriticLoss:{critic_loss}  Reward:{sum(rewards)} ")
        writer.add_scalar("loss/actor_loss",jax.device_get(actor_loss),episode)
        writer.add_scalar("loss/critic_loss:",jax.device_get(critic_loss),episode)
        writer.add_scalar("rewards",sum(jax.device_get(rewards)),episode)
        # writer.add_scalar("advantage",jax.device_get(advantage),episode)

    env.close()
    writer.close()
    wandb.finish()

    return actor_state, critic_state



In [6]:
envs = {"cartPole":"CartPole-v1","acrobot":"Acrobot-v1","mountainCar":"MountainCar-v0"}
gamma = 1.0
episodes = 10000
max_termination = 1000
seed = 0
num_envs = 1
learning_rate = 0.0003

In [9]:
train(envs["mountainCar"],gamma,episodes,max_termination,seed,num_envs,learning_rate)

global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss/actor_loss,▄▂▁▆▇▇█▇██▇▇▇▇█▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▇▇▇
loss/critic_loss:,▇▆▅▄▄▂▂▅▃▄▂▃▂▁▃▃▂▃▂▂▁▂▃▂▁▃▂▁▁▂▁▁▅▂▁▁▁▁▁█
rewards,▁▆▇▇▇██▆▇▇████▇▇▇▇████▇██▇▇██▇██▇▇█████▆
global_step,9999
loss/actor_loss,4.84908
loss/critic_loss:,0.80137
rewards,-62.0


wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`


Moviepy - Building video /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-0.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-0.mp4
Episode:0   ActorLoss:-3.4766998291015625 CriticLoss:0.9999780058860779  Reward:-1000.0 
Moviepy - Building video /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-1.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-1.mp4



Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-1.mp4
Episode:1   ActorLoss:1.723388671875 CriticLoss:1.0002377033233643  Reward:-1000.0 
Episode:2   ActorLoss:0.43910980224609375 CriticLoss:1.000009298324585  Reward:-1000.0 
Episode:3   ActorLoss:-6.335899353027344 CriticLoss:0.9999366402626038  Reward:-1000.0 
Episode:4   ActorLoss:8.066776275634766 CriticLoss:0.9998784065246582  Reward:-1000.0 
Episode:5   ActorLoss:-0.9016666412353516 CriticLoss:0.9999385476112366  Reward:-1000.0 
Episode:6   ActorLoss:-9.008056640625 CriticLoss:1.0001568794250488  Reward:-1000.0 
Episode:7   ActorLoss:-18.988494873046875 CriticLoss:1.0001188516616821  Reward:-1000.0 
Moviepy - Building video /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-8.mp4.
Moviepy - Writing video /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-8.mp4



...

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-7000.mp4
Episode:7000   ActorLoss:-1.1716175079345703 CriticLoss:0.8731068968772888  Reward:-152.0 
Episode:7001   ActorLoss:-2.005544900894165 CriticLoss:0.8831243515014648  Reward:-159.0 
Episode:7002   ActorLoss:-1.2266407012939453 CriticLoss:0.8890246748924255  Reward:-168.0 
Episode:7003   ActorLoss:0.06386715173721313 CriticLoss:0.8721104860305786  Reward:-145.0 
Episode:7004   ActorLoss:-0.8293002843856812 CriticLoss:0.8719105124473572  Reward:-151.0 
Episode:7005   ActorLoss:-0.7833711504936218 CriticLoss:0.8736335039138794  Reward:-153.0 
Episode:7006   ActorLoss:-0.8707468509674072 CriticLoss:0.8756109476089478  Reward:-152.0 
Episode:7007   ActorLoss:-3.0264530181884766 CriticLoss:0.8706402778625488  Reward:-147.0 
Episode:7008   ActorLoss:-1.7093327045440674 CriticLoss:0.8748108744621277  Reward:-151.0 
Episode:7009   ActorLoss:-1.5152746438980103 CriticLoss:0.87

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-8000.mp4
Episode:8000   ActorLoss:-1.2369964122772217 CriticLoss:0.844162106513977  Reward:-108.0 
Episode:8001   ActorLoss:-0.35896700620651245 CriticLoss:0.8765950798988342  Reward:-148.0 
Episode:8002   ActorLoss:-1.8578954935073853 CriticLoss:0.8750340938568115  Reward:-154.0 
Episode:8003   ActorLoss:-2.258283853530884 CriticLoss:0.8790132999420166  Reward:-157.0 
Episode:8004   ActorLoss:-1.4558025598526 CriticLoss:0.8733274340629578  Reward:-148.0 
Episode:8005   ActorLoss:-2.346987247467041 CriticLoss:0.8709321618080139  Reward:-154.0 
Episode:8006   ActorLoss:-0.5462942123413086 CriticLoss:0.8692678809165955  Reward:-156.0 
Episode:8007   ActorLoss:-1.8943767547607422 CriticLoss:0.8717532157897949  Reward:-159.0 
Episode:8008   ActorLoss:-0.41735196113586426 CriticLoss:0.8706449866294861  Reward:-154.0 
Episode:8009   ActorLoss:-1.941080093383789 CriticLoss:0.870518

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-9000.mp4
Episode:9000   ActorLoss:-1.4768134355545044 CriticLoss:0.8730974793434143  Reward:-149.0 
Episode:9001   ActorLoss:-1.1376954317092896 CriticLoss:0.8683357834815979  Reward:-155.0 
Episode:9002   ActorLoss:-2.3157050609588623 CriticLoss:0.8675617575645447  Reward:-159.0 
Episode:9003   ActorLoss:0.1144675612449646 CriticLoss:0.851065993309021  Reward:-118.0 
Episode:9004   ActorLoss:-0.9870502948760986 CriticLoss:0.8736968040466309  Reward:-168.0 
Episode:9005   ActorLoss:-2.5114974975585938 CriticLoss:0.8793404698371887  Reward:-149.0 
Episode:9006   ActorLoss:-0.8564256429672241 CriticLoss:0.8680811524391174  Reward:-155.0 
Episode:9007   ActorLoss:-1.3879289627075195 CriticLoss:0.8685438632965088  Reward:-157.0 
Episode:9008   ActorLoss:-0.06892868876457214 CriticLoss:0.8691335916519165  Reward:-162.0 
Episode:9009   ActorLoss:0.31972450017929077 CriticLoss:0.87

Moviepy - Done !
Moviepy - video ready /notebooks/rl-algos/videos/MountainCar-v0__0__1700039789/rl-video-episode-10000.mp4


(TrainState(step=Array(10000, dtype=int32, weak_type=True), apply_fn=<bound method Module.apply of Actor(
     # attributes
     action_dims = 3
 )>, params=FrozenDict({
     params: {
         Dense_0: {
             bias: Array([ 0.3890526 , -0.24297777,  0.01788891, -0.01455689,  0.08531468,
                     0.4124792 ,  0.18495555, -0.06584055,  0.3103369 ,  0.274437  ,
                     0.2599745 ,  0.14618884,  0.31844646,  0.35240522,  0.17520374,
                     0.37202245], dtype=float32),
             kernel: Array([[-0.5168959 , -1.6821604 , -1.4188243 ,  0.7976215 , -0.12126786,
                      0.8608355 , -0.04409919,  0.12743327, -1.4020491 , -0.5442583 ,
                     -0.6169948 ,  0.06898312,  1.0469633 , -0.7097582 , -0.9452211 ,
                     -0.5557748 ],
                    [ 0.02562904,  0.8810868 ,  1.5217787 , -0.14143288,  2.4037855 ,
                      1.8099678 , -3.1252618 ,  0.3917418 , -0.14016785,  2.1242764 ,
           